In [1]:
# Import libraries and api keys
import hvplot.pandas
import pandas as pd
import requests

from api_keys import geoapify_key

In [2]:
# Load csv file created in KSchultz_WeatherPy
cities_df = pd.read_csv("cities.csv")

# Display first five rows of cities.csv in dataframe
cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,9.90,91,4,0.89,NZ,1698604370
1,1,utrik,11.2278,169.8474,28.20,78,63,8.71,MH,1698604371
2,2,goianesia,-15.3175,-49.1175,34.87,30,52,2.64,BR,1698604371
3,3,mugia,43.1041,-9.2179,12.73,76,100,7.69,ES,1698604371
4,4,fort bragg,35.1390,-79.0060,28.79,50,0,4.63,US,1698604371


In [3]:
# Plot cities.csv on the holoview plot

map_plot = cities_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.6,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

map_plot

C:\Users\Karlschultzcomposer8\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
# Narrow down cities to those suitable for vacation, with weather that is currently mild, sunny and not windy 

cities_reduced_df = cities_df.loc[
    (cities_df["Max Temp"] < 27) & (cities_df["Max Temp"] > 21) \
    & (cities_df["Wind Speed"] < 4.5) \
    & (cities_df["Cloudiness"] == 0),:
]

# Find rows that have cells with empty values and drop entire row if they have one empty cell
cities_reduced_df = cities_reduced_df.dropna()

cities_reduced_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,wailua homesteads,22.0669,-159.3780,25.24,84,0,3.09,US,1698604377
31,31,kasane,-17.8167,25.1500,26.05,41,0,1.03,BW,1698604379
69,69,tsiombe,-25.3000,45.4833,23.59,78,0,2.73,MG,1698604391
77,77,awjilah,29.1081,21.2869,24.14,34,0,3.21,LY,1698604393
100,100,kapa'a,22.0752,-159.3190,25.99,84,0,3.09,US,1698604400
120,120,lihue,21.9789,-159.3672,25.58,83,0,3.09,US,1698604406
227,227,duarte,34.1394,-117.9773,24.80,42,0,2.06,US,1698604245
235,235,at taj,24.2049,23.2857,23.46,25,0,4.27,LY,1698604436
299,299,lompoc,34.6391,-120.4579,24.23,41,0,0.89,US,1698604453
417,417,safford,32.8340,-109.7076,23.19,23,0,2.57,US,1698604485


In [5]:
# Find hotels within suitable cities

hotels_list = cities_reduced_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name"
hotels_list["Hotel Name"] = ""

# Display list of hotels
hotels_list

,City,Country,Lat,Lng,Humidity,Hotel Name
25,wailua homesteads,US,22.0669,-159.3780,84,
31,kasane,BW,-17.8167,25.1500,41,
69,tsiombe,MG,-25.3000,45.4833,78,
77,awjilah,LY,29.1081,21.2869,34,
100,kapa'a,US,22.0752,-159.3190,84,
120,lihue,US,21.9789,-159.3672,83,
227,duarte,US,34.1394,-117.9773,42,
235,at taj,LY,24.2049,23.2857,25,
299,lompoc,US,34.6391,-120.4579,41,
417,safford,US,32.8340,-109.7076,23,


In [6]:
# Find the first hotel located within a radius of 10,000 meters

radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

print("Starting hotel search")

# Use for loop to iterate through hotel_df
for index, row in hotels_list.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Build parameters
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL to get data
    base_url = "https://api.geoapify.com/v2/places"

    # Make API request
    name_address = requests.get(base_url, params=params)
    print(name_address.status_code)
    
    # convert received data into json file
    name_address = name_address.json()
    
    # Grab the first hotel from results
    try:
        hotels_list.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotels_list.loc[index, "Hotel Name"] = "No hotel found"
        
# Display data
hotels_list

Starting hotel search
200
200
200
200
200
200
200
200
200
200
200


,City,Country,Lat,Lng,Humidity,Hotel Name
25,wailua homesteads,US,22.0669,-159.3780,84,Hilton Garden Inn Kauai Wailua Bay
31,kasane,BW,-17.8167,25.1500,41,Kasane Travel Lodge
69,tsiombe,MG,-25.3000,45.4833,78,No hotel found
77,awjilah,LY,29.1081,21.2869,34,No hotel found
100,kapa'a,US,22.0752,-159.3190,84,Pono Kai Resort
120,lihue,US,21.9789,-159.3672,83,Kauai Palms
227,duarte,US,34.1394,-117.9773,42,No hotel found
235,at taj,LY,24.2049,23.2857,25,No hotel found
299,lompoc,US,34.6391,-120.4579,41,Embassy Suites by Hilton Lompoc Central Coast
417,safford,US,32.8340,-109.7076,23,Quality Inn And Suites


In [7]:
# Plot holoview and show hotels on the map

map_plot = hotels_list.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)